## Zadanie domowe: Projektowanie filtru metodą okien

Zauważone w zadaniu z filtracją artefakty są efektem zastosowania filtru idealnego (wycięcia odpowiednich częstotliwości).
Aby je usunąć, należy filtr idealny w dziedzinie przestrzennej przemnożyć przez dwuwymiarowe okno (stworzone na podstawie jednowymiarowego).

1. Zdefiniuj rozmiar filtra (np. 21).

2. Stwórz okno np. Hamminga (`np.Hmming`) o zadanym rozmiarze.

3. Na podstawie tego okna stwórz okno dwuwymiarowe: $F = f^Tf$.
Można to zrealizować prostą instrukcją:
        hanning2d = np.outer(hanning, hanning)

4. Stwórz wzorcowy filtr idealny tak samo jak w poprzednim ćwiczeniu.
Jego rozmiar musi być taki sam jak zdefiniowany w punkcie pierwszym.

5. Generalna zasada jest taka, że należy obliczyć odwrotną transformatę Fouriera filtru idealnego.
Aby zagwarantować jednak dobrą lokalizację i symetryczność tworzonego filtru wykorzystuje się dodatkowe rotacje i przesunięcia.

        FilterFRot = np.rot90(np.fft.fftshift(np.rot90(FilterF, 2)), 2)
        FilterFRot3 = np.dstack((FilterFRot, np.zeros(FilterFRot.shape)))
        FilterFidft = cv2.idft(np.float32(FilterFRot3),
                        flags=cv2.DFT_SCALE | cv2.DFT_COMPLEX_OUTPUT)
        FilterFI = np.rot90(np.fft.fftshift(FilterFidft[:, :, 0]), 2)

6. Aby otrzymać poprawny filtr należy teraz pomnożyć dwuwymiarowe okno ze stworzonym filtrem idealnym w dziedzinie przestrzennej.

7. Stworzony filtr powinien zostać znowu przeniesiony do dziedziny częstotliwości.
Należy jednak rozszerzyć go tak, aby miał taki sam rozmiar jak filtrowany obraz.
Operację tę wykonaj tak samo jak w poprzednim ćwiczeniu (`cv2.copyMakeBorder`).
Oblicz transformatę Fouriera i wykonaj przesunięcie.
Następnie oblicz wartość bezwzględną (dzięki temu filtr nie będzie wpływał na fazę).
Wynikiem jest poprawiony filtr w dziedzinie częstotliwości.

8. Przeprowadź filtrację obrazu *lena.bmp* za pomocą zaprojektowanego filtra.
Wyświetl filtr, oraz wynik filtracji.
Do wyświetlenia filtru możesz użyć przestrzeń częstotliwości wygenerowaną w poprzednim zadaniu (*FSpaceRowsM* i *FSpaceColsM*).

In [ ]:
import cv2
import os
import requests
from matplotlib import pyplot as plt
import numpy as np
import math

url = 'https://raw.githubusercontent.com/vision-agh/poc_sw/master/08_Fourier/'

fileNames = ["lena.bmp"]
for fileName in fileNames:
  if not os.path.exists(fileName):
      r = requests.get(url + fileName, allow_redirects=True)
      open(fileName, 'wb').write(r.content)

In [ ]:
#1
filter_size = 21

#2
hanning_window = np.hanning(filter_size)

#3
hanning2d = np.outer(hanning_window, hanning_window)

#4
def low_pass_filter(size, radius, display_f=True, display_freq=False):
    #3
    f_space_rows = 2 * np.fft.fftshift(np.fft.fftfreq(size))
    f_space_rows_m = np.outer(f_space_rows, np.ones([1, size]))
    f_space_cols = 2 * np.fft.fftshift(np.fft.fftfreq(size))
    f_space_cols_m = np.outer(np.ones([1, size]), f_space_cols)
    freq_r = np.sqrt(np.square(f_space_rows_m) + np.square(f_space_cols_m))

    if display_freq:
        freq_fig = plt.figure()
        ax = freq_fig.add_subplot(projection='3d')
        ax.plot_surface(f_space_rows_m, f_space_cols_m, freq_r, cmap='viridis')
        plt.show()
    #4 
    filter_f = freq_r <= radius
    
    if display_f:
        filter_fig = plt.figure()
        ax = filter_fig.add_subplot(projection='3d')
        ax.plot_surface(f_space_rows_m, f_space_cols_m, filter_f, cmap='viridis')
        plt.show()

    return filter_f, f_space_rows_m, f_space_cols_m

filterF, f_space_rows_m, f_space_cols_m = low_pass_filter(filter_size, 0.1, display_freq=True)

# alternative
# filterF = np.zeros((filter_size, filter_size))
# center = filter_size // 2
# filterF[center, center] = 1


#5
filterFRot = np.rot90(np.fft.fftshift(np.rot90(filterF, 2)), 2)
filterFRot3 = np.dstack((filterFRot, np.zeros(filterFRot.shape)))
filterFidft = cv2.idft(np.float32(filterFRot3), flags=cv2.DFT_SCALE | cv2.DFT_COMPLEX_OUTPUT)
filterFI = np.rot90(np.fft.fftshift(filterFidft[:, :, 0]), 2)

#6
filterF = hanning2d * filterFI

#7
lena = cv2.imread('lena.bmp', cv2.IMREAD_GRAYSCALE)
filterF = cv2.copyMakeBorder(filterF, 0, lena.shape[0] - filter_size, 0, lena.shape[1] - filter_size, cv2.BORDER_CONSTANT, value=0)
filterF = cv2.dft(np.float32(filterF), flags=cv2.DFT_COMPLEX_OUTPUT)
filterF = np.fft.fftshift(filterF)
filterF = cv2.magnitude(filterF[:, :, 0], filterF[:, :, 1])

#8
def display_filter(filter, size):
    f_space_rows =  2 * np.fft.fftshift(np.fft.fftfreq(size))
    f_space_rows_m = np.outer(f_space_rows, np.ones([1, size]))
    f_space_cols = 2 * np.fft.fftshift(np.fft.fftfreq(size))
    f_space_cols_m = np.outer(np.ones([1, size]), f_space_cols)

    fig = plt.figure()
    ax = fig.add_subplot(projection='3d')
    ax.plot_surface(f_space_rows_m, f_space_cols_m, filter, cmap='viridis')
    plt.show()

display_filter(filterF, lena.shape[0])

#9
def filter_img(img, filter):
    filter_f3 = np.repeat(filter[:, :, np.newaxis], 2, axis=2)

    f_img = cv2.dft(np.float32(img), flags=cv2.DFT_COMPLEX_OUTPUT)
    f_img_shift = np.fft.fftshift(f_img, axes=[0,1])
    f_img = f_img_shift * filter_f3
    f_img = np.fft.ifftshift(f_img, axes=[0,1])
    if_img = cv2.idft(f_img, flags = cv2.DFT_SCALE | cv2.DFT_COMPLEX_OUTPUT)
    imgIFFT = cv2.magnitude(if_img[:,:,0], if_img[:,:,1])
    imgIFFT = np.round(imgIFFT).astype(np.uint8)

    fig, axs = plt.subplots(1, 3, figsize=(15, 15))
    axs[0].imshow(img, 'gray')
    axs[1].imshow(imgIFFT, 'gray')
    axs[2].imshow(cv2.absdiff(img, imgIFFT), 'gray')
    
    return imgIFFT

_ = filter_img(lena, filterF)
